## Feature Enrichment

### Historical data enrichment

Pick one client (Site, aka sender_BIC) to do the enrichment as every site will be the same process

In [1]:
site_input_dir = "/tmp/dataset/horizontal_credit_fraud_data/"
site_name = "ZHSZUS33_Bank_1"

In [2]:
import os
import random
import string

import pandas as pd
history_file_name = os.path.join(site_input_dir, site_name,"history.csv" )
df_history = pd.read_csv(history_file_name)
df_history

,Time,Amount,Class,Sender_BIC,Receiver_BIC,UETR,Currency,Beneficiary_BIC,Currency_Country
0,0.0,2.69,0,ZHSZUS33,YSYCESMM,R7PCTKF9R1PVGXRXU9AB3J,AUD,ZNZZAU3M,Australia
1,200.0,3.67,0,ZHSZUS33,ZNZZAU3M,28P261NQ3D4WIZUY4RDXFO,USD,XITXUS33,United States
2,900.0,3.68,0,ZHSZUS33,YMNYFRPP,2XJ54L8ED31VMBC1MYIK8L,AUD,ZNZZAU3M,Australia
3,1700.0,34.09,0,ZHSZUS33,XITXUS33,Y3ZW8BUEF5UTB5LWVNEFPG,GBP,YXRXGB22,United Kingdom
4,2900.0,20.53,0,ZHSZUS33,XITXUS33,FHOWZR8Q77BXKIZHAC0781,USD,ZHSZUS33,United States
...,...,...,...,...,...,...,...,...,...
62558,39325300.0,5.00,0,ZHSZUS33,FBSFCHZH,EC9HYAUYQ3UARN1CMXER1C,AUD,ZNZZAU3M,Australia
62559,39325900.0,61.00,0,ZHSZUS33,ZNZZAU3M,6CT5WHMATEO4Z6UYDECPWR,USD,XITXUS33,United States
62560,39325900.0,1.00,0,ZHSZUS33,ZHSZUS33,GFCUM49U6M2LRN5NBEB9PK,GBP,YXRXGB22,United Kingdom
62561,39327200.0,74.75,0,ZHSZUS33,ZHSZUS33,BLP8GYMXG6JWR104DT3Z8D,USD,ZHSZUS33,United States


In [3]:


history_summary = df_history.groupby('Currency').agg(
                     hist_trans_volume=('UETR', 'count'),
                     hist_total_amount=('Amount', 'sum'),
                     hist_average_amount=('Amount', 'mean')
).reset_index()

history_summary

,Currency,hist_trans_volume,hist_total_amount,hist_average_amount
0,AUD,12572,1094630.75,87.068943
1,CHF,12494,1090937.46,87.316909
2,GBP,12496,1121443.99,89.744237
3,SGD,12460,1121692.50,90.023475
4,USD,12541,1124650.23,89.677875


# Enrich Feature with Currency

In [4]:
import pandas as pd
dataset_names = ["train", "test"]
results = {}

temp_ds_df = {}
temp_resampled_df = {}


for ds_name in dataset_names:
    file_name = os.path.join(site_input_dir, site_name , f"{ds_name}.csv" )
    ds_df  = pd.read_csv(file_name)
    ds_df['Time'] = pd.to_datetime(ds_df['Time'], unit='s')

    # Set the Time column as the index
    ds_df.set_index('Time', inplace=True)
    
    resampled_df = ds_df.resample('1H').agg(
                     trans_volume=('UETR', 'count'),
                     total_amount=('Amount', 'sum'),
                     average_amount=('Amount', 'mean')
                     ).reset_index()
    
    temp_ds_df[ds_name] = ds_df
    temp_resampled_df[ds_name] = resampled_df
    


In [ ]:
for ds_name in dataset_names:
        
    ds_df = temp_ds_df[ds_name]
    resampled_df = temp_resampled_df[ds_name]
    
    c_df = ds_df[['Currency']].resample('1H').agg({'Currency': 'first'}).reset_index()
    # Add Currency_Country to the resampled data by joining with the original DataFrame
    resampled_df2 = pd.merge(resampled_df, 
                            c_df,
                            on='Time'
                            )
    resampled_df3 = pd.merge(resampled_df2, 
                             history_summary,
                             on='Currency'
                            )
    resampled_df4 = resampled_df3.copy()
    resampled_df4['x2_y1'] = resampled_df4['average_amount']/resampled_df4['hist_trans_volume']
    
    ds_df = ds_df.sort_values('Time')
    resampled_df4 = resampled_df4.sort_values('Time')
    merged_df = pd.merge_asof(ds_df, resampled_df4, on='Time' )
    
    merged_df = merged_df.drop(columns=['Currency_y']).rename(columns={'Currency_x': 'Currency'})

    
    results[ds_name] = merged_df
    
    
    

print(results)

# Enrich feature for beneficiary country

In [6]:

history_summary2 = df_history.groupby('Beneficiary_BIC').agg(
                     hist_trans_volume=('UETR', 'count'),
                     hist_total_amount=('Amount', 'sum'),
                     hist_average_amount=('Amount', 'mean')
).reset_index()

history_summary2

,Beneficiary_BIC,hist_trans_volume,hist_total_amount,hist_average_amount
0,FBSFCHZH,12494,1090937.46,87.316909
1,HCBHSGSG,12460,1121692.50,90.023475
2,XITXUS33,6211,572653.93,92.199957
3,YXRXGB22,12496,1121443.99,89.744237
4,ZHSZUS33,6330,551996.30,87.203207
5,ZNZZAU3M,12572,1094630.75,87.068943


In [ ]:
import pandas as pd
dataset_names = ["train", "test"]
results2 = {}
for ds_name in dataset_names:
    ds_df = temp_ds_df[ds_name]
    resampled_df = temp_resampled_df[ds_name]
    
    c_df = ds_df[['Beneficiary_BIC']].resample('1H').agg({'Beneficiary_BIC': 'first'}).reset_index()
    
    # Add Beneficiary_BIC to the resampled data by joining with the original DataFrame
    resampled_df2 = pd.merge(resampled_df, 
                            c_df,
                            on='Time'
                            )
    
    resampled_df3 = pd.merge(resampled_df2, 
                             history_summary2,
                             on='Beneficiary_BIC'
                            )
    
    
    resampled_df4 = resampled_df3.copy()
    resampled_df4['x3_y2'] = resampled_df4['average_amount']/resampled_df4['hist_trans_volume']
   
    ds_df = ds_df.sort_values('Time')
    resampled_df4 = resampled_df4.sort_values('Time')

    merged_df2 = pd.merge_asof(ds_df, resampled_df4, on='Time' )
    merged_df2 = merged_df2.drop(columns=['Beneficiary_BIC_y']).rename(columns={'Beneficiary_BIC_x': 'Beneficiary_BIC'})
    
    
    results2[ds_name] = merged_df2

print(results2)

In [8]:
final_results = {}
for name in results:
    df = results[name]
    df2 = results2[name]
    df3 = df2[["Time", "Beneficiary_BIC", "x3_y2"]].copy()
    df4 = pd.merge(df, df3, on=['Time', 'Beneficiary_BIC'])
    final_results[name] = df4

    
for name in final_results:
    site_dir = os.path.join(site_input_dir, site_name)
    os.makedirs(site_dir, exist_ok=True)
    enrich_file_name = os.path.join(site_dir, f"{name}_enrichment.csv")
    print(enrich_file_name)
    final_results[name].to_csv(enrich_file_name) 
    
final_results["train"]

/tmp/dataset/horizontal_credit_fraud_data/ZHSZUS33_Bank_1/train_enrichment.csv
/tmp/dataset/horizontal_credit_fraud_data/ZHSZUS33_Bank_1/test_enrichment.csv


,Time,Class,Amount,Sender_BIC,Receiver_BIC,UETR,Currency,Beneficiary_BIC,Currency_Country,trans_volume,total_amount,average_amount,hist_trans_volume,hist_total_amount,hist_average_amount,x2_y1,x3_y2
0,1971-04-01 04:30:00,0,348.06,ZHSZUS33,YXRXGB22,MV2B0B0S1NUTY8OCOEQ2QE,USD,XITXUS33,United States,4,422.18,105.545,12541,1124650.23,89.677875,0.008416,0.016993
1,1971-04-01 04:35:00,0,2.69,ZHSZUS33,YMNYFRPP,CQD9INGI7GJATKWRK0D44Z,SGD,HCBHSGSG,Singapore,4,422.18,105.545,12541,1124650.23,89.677875,0.008416,0.016993
2,1971-04-01 04:40:00,0,16.63,ZHSZUS33,XITXUS33,IJXYXLV8SF72RU3MRSJ542,CHF,FBSFCHZH,Switzerland,4,422.18,105.545,12541,1124650.23,89.677875,0.008416,0.016993
3,1971-04-01 04:51:40,0,54.80,ZHSZUS33,XITXUS33,B1850ZUIHTMT61N7HMIZYM,CHF,FBSFCHZH,Switzerland,4,422.18,105.545,12541,1124650.23,89.677875,0.008416,0.016993
4,1971-04-01 05:16:40,0,31.96,ZHSZUS33,ZHSZUS33,4BBLS9B31LWHZFF17RODX1,GBP,YXRXGB22,United Kingdom,4,292.64,73.160,12496,1121443.99,89.744237,0.005855,0.005855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40804,1972-03-10 19:01:40,0,12.99,ZHSZUS33,WPUWDEFF,EBY8SA8UZOWNNJ2X7OUBZ2,USD,XITXUS33,United States,1,12.99,12.990,12541,1124650.23,89.677875,0.001036,0.002091
40805,1972-03-10 21:30:00,0,52.34,ZHSZUS33,YXRXGB22,3D4772259A6PY7Q7XVJ302,GBP,YXRXGB22,United Kingdom,2,272.62,136.310,12496,1121443.99,89.744237,0.010908,0.010908
40806,1972-03-10 21:36:40,0,220.28,ZHSZUS33,YSYCESMM,Z5VK0S69KASH3B82M6W5XV,USD,ZHSZUS33,United States,2,272.62,136.310,12496,1121443.99,89.744237,0.010908,0.010908
40807,1972-03-10 22:30:00,0,60.50,ZHSZUS33,YXRXGB22,HA4WJAB98YR8M9FIE0C2A1,USD,XITXUS33,United States,2,85.29,42.645,12541,1124650.23,89.677875,0.003400,0.006866


In [9]:
! tree {site_input_dir}

/tmp/dataset/horizontal_credit_fraud_data/
├── FBSFCHZH_Bank_6
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── HCBHSGSG_Bank_9
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── history.csv
├── SHSHKHH1_Bank_2
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── test.csv
├── train.csv
├── WPUWDEFF_Bank_4
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── XITXUS33_Bank_10
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── YMNYFRPP_Bank_5
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── YSYCESMM_Bank_7
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── YXRXGB22_Bank_3
│   ├── history.csv
│   ├── test.csv
│   └── train.csv
├── ZHSZUS33_Bank_1
│   ├── history.csv
│   ├── test.csv
│   ├── test_enrichment.csv
│   ├── train.csv
│   └── train_enrichment.csv
└── ZNZZAU3M_Bank_8
    ├── history.csv
    ├── test.csv
    └── train.csv

10 directories, 35 files


In [ ]:
ls -al  /tmp/dataset/horizontal_credit_fraud_data/ZHSZUS33_Bank_1/

In [ ]:
! find /tmp/dataset/horizontal_credit_fraud_data/ZHSZUS33_Bank_1/ -exec wc -l {} \;

Let's go back to the [XGBoost Notebook](./xgboost.ipynb)